<a href="https://colab.research.google.com/github/ttn9171/Portfolio/blob/main/Data%20merging/UC2_Merge_Email_Campaigns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set up Connection

In [ ]:
!apt-get update
!apt-get install -y -q curl gnupg

!curl -fsSL https://packages.cloud.google.com/apt/doc/apt-key.gpg | gpg --dearmor -o /usr/share/keyrings/cloud.google.gpg
!echo "deb [signed-by=/usr/share/keyrings/cloud.google.gpg] https://packages.cloud.google.com/apt gcsfuse-jammy main" | tee /etc/apt/sources.list.d/gcsfuse.list
!apt-get update

!apt-get install -y -q gcsfuse

from google.colab import auth
auth.authenticate_user()

import os
project_id = 'capstone-aldo'
os.environ['Aldo_Capstone'] = project_id

from google.cloud import storage

client = storage.Client(project=project_id)
bucket = 'mma-capstone'

os.makedirs('/content/gcs', exist_ok=True)
!gcsfuse {bucket} /content/gcs

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://packages.cloud.google.com/apt gcsfuse-jammy InRelease
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
R

## Merge Rev Attribution with Email Campaign and Email Campaign Categories


In [ ]:
import pandas as pd

# --------- CONFIG ---------
email_campaigns_cleaned_path = "/content/gcs/emarsys/email_campaigns.csv"
revenue_att_cleaned_path = "/content/gcs/emarsys/revenue_attribution.csv"
email_campaign_categories_path = "/content/gcs/emarsys/email_campaign_categories.csv"
OUTPUT_PATH = "/content/gcs/emarsys/revenue_attribution_updated.csv"

# --------- PREPROCESSING ---------
REQUIRED_COLUMNS_EMAIL_CAMPAIGNS = [
    'id', 'name', 'category_id', 'parent_campaign_id',
    'type', 'program_id', 'event_time', 'evnt_year', 'evnt_mnth', 'bann'
]

FILTER_YEARS = [2023, 2024]
FILTER_BRAND = "ALDO"

def load_and_process_email_campaigns(path):
    df = pd.read_csv(path, usecols=REQUIRED_COLUMNS_EMAIL_CAMPAIGNS)
    df = df[(df['bann'] == FILTER_BRAND) & (df['evnt_year'].isin(FILTER_YEARS))]
    df = df.drop(columns='bann')

    df = df.drop_duplicates(
        ['id', 'name', 'category_id', 'parent_campaign_id', 'program_id']
    )

    df['event_time'] = pd.to_datetime(df['event_time'])
    df['id'] = df['id'].astype(int)
    df['category_id'] = df['category_id'].astype(int)
    return df

def assign_closest_category_id(df):
    df_with_cat = df[df['category_id'] != 0].copy()
    df_without_cat = df[df['category_id'] == 0].copy()

    def get_closest(row):
        matches = df_with_cat[df_with_cat['id'] == row['id']]
        if matches.empty:
            return 0
        matches = matches.copy()
        matches['time_diff'] = (matches['event_time'] - row['event_time']).abs()
        return matches.loc[matches['time_diff'].idxmin(), 'category_id']

    df.loc[df['category_id'] == 0, 'category_id'] = df_without_cat.apply(get_closest, axis=1)
    return df

def preprocess_email_campaigns(path):
    df = load_and_process_email_campaigns(path)
    df = assign_closest_category_id(df)
    return df

def preprocess_revenue_attribution(path):
    df = pd.read_csv(path)
    df = df[(df['treatment_channel'] == 'email') & (df['bann'] == 'ALDO')]
    return df

# --------- MERGE ---------

def merge_email_campaign_revenue_att(email_campaigns_cleaned, email_campaign_categories, revenue_att_cleaned):
    # First merge: attach email category name into to email campaigns
    campaigns_with_categories = email_campaigns_cleaned.merge(
        email_campaign_categories,
        how='left',
        left_on='category_id',
        right_on='id',
        suffixes=('_email_campaign', '_email_category')
    )

    campaigns_subset = campaigns_with_categories[['id_email_campaign', 'name_email_campaign', 'name_email_category']]

    # Second merge: left join with revenue attribution
    final_merged = revenue_att_cleaned.merge(
        campaigns_subset,
        how='left',
        left_on='treatment_campaign_id',
        right_on='id_email_campaign'
    )

    final_merged.drop(columns=['id_email_campaign'], inplace=True)

    final_merged.drop_duplicates(inplace=True)

    final_merged.to_csv(OUTPUT_PATH, index=False)
    print(f"Processing completed. Output saved to: {OUTPUT_PATH}")

    return final_merged

# --------- PIPELINE ---------

def rev_att_email_campaign_pipeline():

    email_campaigns_cleaned = preprocess_email_campaigns(email_campaigns_cleaned_path)
    revenue_att_cleaned = preprocess_revenue_attribution(revenue_att_cleaned_path)
    email_campaign_categories = pd.read_csv(email_campaign_categories_path)

    rev_att_final_df = merge_email_campaign_revenue_att(
        email_campaigns_cleaned,
        email_campaign_categories,
        revenue_att_cleaned
    )

# --------- RUN ---------
if __name__ == "__main__":
    rev_att_email_campaign_pipeline()


Processing completed. Output saved to: /content/gcs/emarsys/revenue_attribution_updated.csv


In [ ]:
rev_att_final_df = pd.read_csv('/content/gcs/emarsys/revenue_attribution_updated.csv')
rev_att_final_df.head()

<ipython-input-3-3bfc17d036d2>:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  rev_att_final_df = pd.read_csv('/content/gcs/emarsys/revenue_attribution_updated.csv')


,customer_id,contact_id,event_time,loaded_at,order_id,partitiontime,meta_job_id,meta_origin,meta_originvers,meta_originsrc,...,treatment_email_launch_id,treatment_event_time,treatment_attributed_amount,treatment_reason_type,treatment_reason_event_time,bann,evnt_year,evnt_mnth,name_email_campaign,name_email_category
0,1030649091,868305899,2023-09-29,2023-10-01,391841233,2023-09-29 00:00:00.0,glue_jr_d36d53046e6bf17af69f197da5b98cd9da1b46...,EMARSYS,NaN,glue_revenue_attribution,...,145592,2023-09-26 15:29:27.0,59.98,click,2023-09-26 21:26:27.0,ALDO,2023,9,aldo-ca-w-2023-wk16.1-new-arrival-bootshop-ank...,newarrivals
1,1030649091,868305899,2023-09-29,2023-10-01,391841233,2023-09-29 00:00:00.0,glue_jr_d36d53046e6bf17af69f197da5b98cd9da1b46...,EMARSYS,NaN,glue_revenue_attribution,...,145592,2023-09-26 15:29:27.0,59.98,click,2023-09-26 21:26:27.0,ALDO,2023,9,Copy of aldo-ca-w-2023-wk16.1-new-arrival-boot...,newarrivals
2,1030649091,23121020,2023-09-29,2023-10-01,391855141,2023-09-29 00:00:00.0,glue_jr_d36d53046e6bf17af69f197da5b98cd9da1b46...,EMARSYS,NaN,glue_revenue_attribution,...,122899,2023-09-29 21:32:03.0,150.00,click,2023-09-29 21:39:12.0,ALDO,2023,9,PROD - ALDO_CA - EN - Welcome - Explicit Consent,WELCOME_EXPLICIT_CONSENT
3,1030649091,23121020,2023-09-29,2023-10-01,391855141,2023-09-29 00:00:00.0,glue_jr_d36d53046e6bf17af69f197da5b98cd9da1b46...,EMARSYS,NaN,glue_revenue_attribution,...,122899,2023-09-29 21:32:03.0,150.00,click,2023-09-29 21:39:12.0,ALDO,2023,9,PROD - ALDO_CA - EN - Automated - Welcome Expl...,WELCOME_JOURNEY
4,1030649091,23121020,2023-09-29,2023-10-01,391855141,2023-09-29 00:00:00.0,glue_jr_d36d53046e6bf17af69f197da5b98cd9da1b46...,EMARSYS,NaN,glue_revenue_attribution,...,122899,2023-09-29 21:32:03.0,150.00,click,2023-09-29 21:39:12.0,ALDO,2023,9,PROD - ALDO_CA - EN - Automated - Welcome Expl...,WELCOME_EXPLICIT_CONSENT


In [ ]:
rev_att_final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453158 entries, 0 to 453157
Data columns (total 27 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   customer_id                  453158 non-null  int64  
 1   contact_id                   453158 non-null  int64  
 2   event_time                   453158 non-null  object 
 3   loaded_at                    453158 non-null  object 
 4   order_id                     453144 non-null  object 
 5   partitiontime                453158 non-null  object 
 6   meta_job_id                  453158 non-null  object 
 7   meta_origin                  453158 non-null  object 
 8   meta_originvers              0 non-null       float64
 9   meta_originsrc               453158 non-null  object 
 10  meta_ingestdt                453158 non-null  object 
 11  item_id                      453158 non-null  int64  
 12  item_price                   453158 non-null  float64
 13 

In [ ]:
rev_att_final_df.isnull().sum()

,0
customer_id,0
contact_id,0
event_time,0
loaded_at,0
order_id,14
partitiontime,0
meta_job_id,0
meta_origin,0
meta_originvers,453158
meta_originsrc,0


## Merge Updated Rev Attribution and CAR

In [ ]:
import pandas as pd
import numpy as np


# --------- CONFIG ---------

RETURN_0_PATH = "/content/gcs/Feb_data_request_Transactions/cleaned/retn_flag_0.csv"
RETURN_1_PATH = "/content/gcs/Feb_data_request_Transactions/cleaned/retn_flag_1.csv"
REVENUE_PATH = "/content/gcs/emarsys/revenue_attribution_updated.csv"
RETURN_0_REV_OUTPUT_PATH = "/content/gcs/Data Preprocessed/retn_flag_0_rev_att_preprocessed.csv"
RETURN_1_REV_OUTPUT_PATH = "/content/gcs/Data Preprocessed/retn_flag_1_rev_att_preprocessed.csv"

# --------- MERGE ---------

def merge_CAR_with_revenue_att(car_path, revenue_path, output_path):
    car_df = pd.read_csv(car_path)
    revenue_df = pd.read_csv(revenue_path, dtype={'order_id': str})

    car_df['ecom_trsn_sid'] = car_df['ecom_trsn_sid'].astype(str).str.replace('.0', '', regex=False)

    revenue_cols_to_keep = [
        'order_id', 'treatment_campaign_id', 'name_email_campaign',
        'treatment_id', 'name_email_category', 'treatment_email_launch_id',
        'treatment_attributed_amount'
    ]
    revenue_df = revenue_df[revenue_cols_to_keep]

    # Perform left merge
    merged_df = car_df.merge(
        revenue_df,
        how='left',
        left_on=['ecom_trsn_sid'],
        right_on=['order_id']
    )

    merged_df.drop(columns=['order_id'], inplace=True)
    merged_df.drop_duplicates(inplace=True)

    # Deal with NAs
    merged_df = merged_df.dropna(subset=['ecom_trsn_sid'])

    int_columns = ['treatment_campaign_id', 'treatment_id', 'treatment_email_launch_id']
    for col in int_columns:
        merged_df[col] = merged_df[col].fillna(0).astype(int)

    cat_columns = ['name_email_campaign', 'name_email_category']
    for col in cat_columns:
        merged_df[col] = merged_df[col].fillna('notapplicable')

    merged_df['treatment_attributed_amount'] = merged_df['treatment_attributed_amount'].fillna(0)

    merged_df.to_csv(output_path, index=False)
    print(f"Processing completed. Output saved to: {output_path}")

    return merged_df

# --------- EXECUTE ---------

if __name__ == "__main__":
    merge_CAR_with_revenue_att(RETURN_0_PATH, REVENUE_PATH, RETURN_0_REV_OUTPUT_PATH)
    merge_CAR_with_revenue_att(RETURN_1_PATH, REVENUE_PATH, RETURN_1_REV_OUTPUT_PATH)


Processing completed. Output saved to: /content/gcs/Data Preprocessed/retn_flag_0_rev_att_preprocessed.csv
Processing completed. Output saved to: /content/gcs/Data Preprocessed/retn_flag_1_rev_att_preprocessed.csv


In [ ]:
df_ret0 = pd.read_csv('/content/gcs/Data Preprocessed/retn_flag_0_rev_att_preprocessed.csv')

In [ ]:
df_ret0[df_ret0['treatment_campaign_id'] != 0]

,artl_gnrc_id,retn_flag,disc_offr_id,ecom_trsn_sid,ordr_net_sale_qty,net_prc_locl,pos_disc_amt_locl,ordr_net_sale_retl_amt_locl,ordr_net_sale_cost_amt_locl,trans_dt,price_group,orgnl_retl_prc_locl,bann_cd,treatment_campaign_id,name_email_campaign,treatment_id,name_email_category,treatment_email_launch_id,treatment_attributed_amount
897992,13343733,0,notprovided,395223647,1,15.0,0.0,15.0,4.44,2024-01-24,Regular,15.0,ALDO_CA,334021,aldo-w-2024-wk24.1-newarrivals-new-arrival-han...,1051134,newarrivals,408510,49.98
897996,13343733,0,LWE_CA,395074176,4,15.0,-9.0,51.0,17.76,2024-01-18,Promo Reg,15.0,ALDO_CA,103465,PROD - ALDO_CA - EN - Aldo Crew Loyalty - Welc...,25583,CREW_WELCOME_INITIAL,122955,68.00
897997,13343733,0,LWE_CA,395074176,4,15.0,-9.0,51.0,17.76,2024-01-18,Promo Reg,15.0,ALDO_CA,103465,PROD - ALDO_CA - EN - Loyalty - Aldo Crew Welc...,25583,LOYALTY_JOURNEY,122955,68.00
897998,13343733,0,LWE_CA,395074176,4,15.0,-9.0,51.0,17.76,2024-01-18,Promo Reg,15.0,ALDO_CA,103465,PROD - ALDO_CA - EN - Loyalty - Aldo Crew Welc...,25583,CREW_WELCOME_INITIAL,122955,68.00
897999,13343733,0,LWE_CA,395074176,4,15.0,-9.0,51.0,17.76,2024-01-18,Promo Reg,15.0,ALDO_CA,103465,Distributed on 2023-09-13 15:54:57.165 - AL...,25583,CREW_WELCOME_INITIAL,122955,68.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4191349,13620649,0,notprovided,392561510,1,98.0,0.0,98.0,15.34,2023-11-02,Regular,98.0,ALDO_US,183978,aldo-us-w-2023-wk19.1-promo-bogo-40-off-launch...,57982,promo,219239,302.00
4191367,13620649,0,notprovided,392365034,1,98.0,0.0,98.0,15.34,2023-10-24,Regular,98.0,ALDO_US,156311,aldo-us-w-2023-wk17.2-promo-bogo-40-off-last-d...,520394,promo,200106,156.80
4191370,13620649,0,notprovided,392369957,1,98.0,0.0,98.0,15.34,2023-10-24,Regular,98.0,ALDO_US,156311,aldo-us-w-2023-wk17.2-promo-bogo-40-off-last-d...,1689862,promo,200106,128.00
4191380,13620649,0,notprovided,392320496,1,98.0,0.0,98.0,15.33,2023-10-22,Regular,98.0,ALDO_US,159721,aldo-us-w-2023-wk18.1-new-arrival-newarrivals-...,1827854,newarrivals,190907,250.00


In [ ]:
df_ret0.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3917669 entries, 0 to 4191401
Data columns (total 19 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   artl_gnrc_id                 int64  
 1   retn_flag                    int64  
 2   disc_offr_id                 object 
 3   ecom_trsn_sid                object 
 4   ordr_net_sale_qty            int64  
 5   net_prc_locl                 float64
 6   pos_disc_amt_locl            float64
 7   ordr_net_sale_retl_amt_locl  float64
 8   ordr_net_sale_cost_amt_locl  float64
 9   trans_dt                     object 
 10  price_group                  object 
 11  orgnl_retl_prc_locl          float64
 12  bann_cd                      object 
 13  treatment_campaign_id        int64  
 14  name_email_campaign          object 
 15  treatment_id                 int64  
 16  name_email_category          object 
 17  treatment_email_launch_id    int64  
 18  treatment_attributed_amount  float64
dtypes: fl

In [ ]:
df_ret0.isnull().sum()

,0
artl_gnrc_id,0
retn_flag,0
disc_offr_id,1691126
ecom_trsn_sid,0
ordr_net_sale_qty,0
net_prc_locl,0
pos_disc_amt_locl,0
ordr_net_sale_retl_amt_locl,0
ordr_net_sale_cost_amt_locl,0
trans_dt,0


## Merge GA, CAR, and REV_ATT

In [ ]:
import pandas as pd

# --------- CONFIG ---------
GA_SESSIONS_PATH = "/content/gcs/GA/ga_sample3/final/ALL_ga_sample3_preprocessed.csv"
NON_RETURN_TRANSACTIONS_PATH = "/content/gcs/Data Preprocessed/retn_flag_0_rev_att_preprocessed.csv"
RETURN_TRANSACTIONS_PATH = "/content/gcs/Data Preprocessed/retn_flag_1_rev_att_preprocessed.csv"
OUTPUT_PATH = "/content/gcs/Data Preprocessed/GA_ecom_rev.csv"

# --------- PIPELINE FUNCTIONS ---------

def load_ga_sessions(path):
    return pd.read_csv(path, dtype={'transaction_id': str, 'product_sku': str}, low_memory=False)

def load_ecom_data(file_path):
    df = pd.read_csv(file_path, dtype={'ecom_trsn_sid': str, 'artl_gnrc_id': str})
    df['ecom_trsn_sid'] = df['ecom_trsn_sid'].str.replace(".0", "", regex=False)
    df.drop_duplicates(inplace=True)

    return df

def merge_non_return_transactions(ga_df, non_return_df):
    merged = ga_df.merge(
        non_return_df[['ecom_trsn_sid', 'artl_gnrc_id', 'retn_flag', 'disc_offr_id', 'orgnl_retl_prc_locl',
                       'price_group', 'pos_disc_amt_locl', 'ordr_net_sale_retl_amt_locl',
                       'ordr_net_sale_cost_amt_locl', 'trans_dt',
                       'treatment_campaign_id', 'name_email_campaign',
                       'treatment_id', 'name_email_category',
                       'treatment_email_launch_id','treatment_attributed_amount'
                       ]],
        how='left',
        left_on=['transaction_id', 'product_sku'],
        right_on=['ecom_trsn_sid', 'artl_gnrc_id']

    )
    merged['retn_flag'] = merged['retn_flag'].fillna(0).astype(int)
    return merged.drop_duplicates()

def merge_return_transactions(ga_df, return_df):
    merged = return_df.merge(
        ga_df[['transaction_id', 'product_sku', 'fullvisitor_id']],
        how='left',
        left_on=['ecom_trsn_sid', 'artl_gnrc_id'],
        right_on=['transaction_id', 'product_sku']

    )
    return merged.dropna(subset=['fullvisitor_id'])

def combine_datasets(non_return_df, return_df):
    return pd.concat([non_return_df, return_df], ignore_index=True)

def clean_and_impute(df):

    df['disc_offr_id'] = df['disc_offr_id'].fillna('notprovided')

    df.loc[(df['retn_flag'] == 1) & (df['date'].isna()) & (df['trans_dt'].notna()), 'date'] = df['trans_dt']
    df.loc[(df['retn_flag'] == 1) & (df['country'].isna()) & (df['bann_cd'].notna()),'country'] = df['bann_cd']
    df.loc[(df['retn_flag'] == 1) & (df['product_revenue'].isna()) & (df['ordr_net_sale_retl_amt_locl'].notna()),
           'product_revenue'] = df['ordr_net_sale_retl_amt_locl']
    df.loc[(df['retn_flag'] == 0) & (df['trans_dt'].isna()) & (df['date'].notna()), 'trans_dt'] = df['date']
    df.loc[(df['retn_flag'] == 1) & (df['quantity'].isna()), 'quantity'] = -1

    df['price_group'] = df['price_group'].fillna('Unknown')
    df['pos_disc_amt_locl'] = df['pos_disc_amt_locl'].fillna(0)
    df['ordr_net_sale_retl_amt_locl'] = df['ordr_net_sale_retl_amt_locl'].fillna(df['product_revenue'])

    df['orgnl_retl_prc_locl'] = df.groupby(['product_sku', 'quantity'])['orgnl_retl_prc_locl'].transform(
        lambda x: x.fillna(x.median()))
    df['ordr_net_sale_cost_amt_locl'] = df.groupby(['product_sku', 'quantity'])['ordr_net_sale_cost_amt_locl'].transform(
        lambda x: x.fillna(x.median()))

    int_columns = ['treatment_campaign_id', 'treatment_id', 'treatment_email_launch_id']
    for col in int_columns:
        df[col] = df[col].fillna(0).astype(int)

    cat_columns = ['name_email_campaign', 'name_email_category']
    for col in cat_columns:
        df[col] = df[col].fillna('notapplicable')

    df['treatment_attributed_amount'] = df['treatment_attributed_amount'].fillna(0)

    df.drop(columns=['bann_cd', 'ordr_net_sale_qty', 'net_prc_locl', 'ecom_trsn_sid', 'artl_gnrc_id'], inplace=True, errors='ignore')

    return df.drop_duplicates()

def save_data(df, output_path):
    df.to_csv(output_path, index=False)
    print(f"✅ Data saved to: {output_path}")

# --------- PIPELINE ---------

def merge_ca_car_pipeline():
    print("Starting pipeline...")

    ga_df = load_ga_sessions(GA_SESSIONS_PATH)
    ecom_nonreturn = load_ecom_data(NON_RETURN_TRANSACTIONS_PATH)
    ecom_return = load_ecom_data(RETURN_TRANSACTIONS_PATH)

    ga_nonreturn_merged = merge_non_return_transactions(ga_df, ecom_nonreturn)
    return_with_visitors = merge_return_transactions(ga_df, ecom_return)

    combined_df = combine_datasets(ga_nonreturn_merged, return_with_visitors)
    cleaned_df = clean_and_impute(combined_df)

    save_data(cleaned_df, OUTPUT_PATH)
    print("Pipeline complete.")

# --------- RUN ---------
if __name__ == "__main__":
    merge_ca_car_pipeline()


Starting pipeline...
✅ Data saved to: /content/gcs/Data Preprocessed/GA_ecom_rev.csv
Pipeline complete.


In [ ]:
df = pd.read_csv('/content/gcs/Data Preprocessed/GA_ecom_rev.csv')

<ipython-input-60-8213b581fb00>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/gcs/Data Preprocessed/GA_ecom_rev.csv')


In [ ]:
pd.set_option('display.max_columns', None)
df.head()

,date,fullvisitor_id,transaction_id,product_sku,quantity,product_revenue,country,retn_flag,disc_offr_id,orgnl_retl_prc_locl,price_group,pos_disc_amt_locl,ordr_net_sale_retl_amt_locl,ordr_net_sale_cost_amt_locl,trans_dt,treatment_campaign_id,name_email_campaign,treatment_id,name_email_category,treatment_email_launch_id,treatment_attributed_amount
0,2021-01-24,=5256685714421882478=,368292564,13066104,1.0,18.88,ALDO_CA,0,notprovided,NaN,Unknown,0.0,18.88,NaN,2021-01-24,0,notapplicable,0,notapplicable,0,0.0
1,2021-01-07,=355653551763437664=,367659729,13097479,1.0,74.98,ALDO_CA,0,notprovided,110.0,Unknown,0.0,74.98,27.04,2021-01-07,0,notapplicable,0,notapplicable,0,0.0
2,2021-01-26,=7792424802434576941=,368380343,13067038,1.0,29.98,ALDO_CA,0,notprovided,60.0,Markdown,0.0,29.98,18.27,2021-02-01,0,notapplicable,0,notapplicable,0,0.0
3,2021-01-19,=5083839754936321960=,368116883,12997328,1.0,15.28,ALDO_CA,0,notprovided,25.0,Unknown,0.0,15.28,6.38,2021-01-19,0,notapplicable,0,notapplicable,0,0.0
4,2021-01-21,=5631864039874872619=,368213547,12881879,1.0,39.98,ALDO_CA,0,notprovided,80.0,Unknown,0.0,39.98,21.07,2021-01-21,0,notapplicable,0,notapplicable,0,0.0


In [ ]:
df[(df['name_email_category']!= 'notapplicable')]

,date,fullvisitor_id,transaction_id,product_sku,quantity,product_revenue,country,retn_flag,disc_offr_id,orgnl_retl_prc_locl,price_group,pos_disc_amt_locl,ordr_net_sale_retl_amt_locl,ordr_net_sale_cost_amt_locl,trans_dt,treatment_campaign_id,name_email_campaign,treatment_id,name_email_category,treatment_email_launch_id,treatment_attributed_amount
907769,2023-10-03,=14491412484945813043=,391923932,13679853,1.0,220.00,ALDO_CA,0,notprovided,220.0,Regular,0.00,220.00,48.86,2023-10-03,132053,aldo-ca-w-2023-wk16.2-new-arrival-bootshop-tal...,387618,newarrivals,157906,280.00
907770,2023-10-03,=14491412484945813043=,391923932,13679853,1.0,220.00,ALDO_CA,0,notprovided,220.0,Regular,0.00,220.00,48.86,2023-10-03,132053,FIFTH GRID TEST CA,387618,newarrivals,157906,280.00
907779,2023-10-24,=12510743919872426475=,392369726,13618354,1.0,69.98,ALDO_CA,0,notprovided,135.0,Markdown,0.00,69.98,37.82,2023-10-24,55586,PROD - ALDO_CA - EN - Transactional - Shipping...,59215,ORDER_SHIPPING_CONFIRMATION,66822,69.98
907780,2023-10-24,=12510743919872426475=,392369726,13618354,1.0,69.98,ALDO_CA,0,notprovided,135.0,Markdown,0.00,69.98,37.82,2023-10-24,55586,PROD - ALDO_CA - EN - Transactional - Shipping...,59215,POST_PURCHASE,66822,69.98
907781,2023-10-24,=12510743919872426475=,392369726,13618354,1.0,69.98,ALDO_CA,0,notprovided,135.0,Markdown,0.00,69.98,37.82,2023-10-24,55586,Distributed on 2023-08-10 17:59:11.891 - ALDO_...,59215,ORDER_SHIPPING_CONFIRMATION,66822,69.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2064861,2023-11-21,=8506971107033031128=,392841767,13618407,-1.0,-64.98,ALDO_US,1,notprovided,80.0,Promo MD,0.00,-64.98,-10.69,2023-11-21,192948,aldo-us-w-2023-wk18.1-promo-bogo-40-last-day-f...,1331574,promo,236931,199.90
2064888,2023-12-31,=8140274551719415669=,393510368,13620649,-1.0,-55.99,ALDO_US,1,b2get20-us,90.0,Promo MD,13.99,-55.99,-15.36,2023-12-31,232641,aldo-w-2023-wk20.2-sale-black-friday-last-day-...,1952260,sale,275271,355.93
2064897,2023-12-07,=847116523410843872=,392561510,13620649,-1.0,-98.00,ALDO_US,1,notprovided,90.0,Promo MD,0.00,-98.00,-15.33,2023-12-07,183978,aldo-us-w-2023-wk19.1-promo-bogo-40-off-launch...,57982,promo,219239,302.00
2064898,2023-12-07,=847116523410843872=,392561510,13620649,-1.0,-98.00,ALDO_US,1,notprovided,90.0,Promo MD,0.00,-98.00,-15.33,2023-12-07,183978,aldo-us-w-2023-wk19.1-promo-bogo-40-off-launch...,57982,promo,219239,302.00


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2064909 entries, 0 to 2064908
Data columns (total 21 columns):
 #   Column                       Dtype  
---  ------                       -----  
 0   date                         object 
 1   fullvisitor_id               object 
 2   transaction_id               int64  
 3   product_sku                  object 
 4   quantity                     float64
 5   product_revenue              float64
 6   country                      object 
 7   retn_flag                    int64  
 8   disc_offr_id                 object 
 9   orgnl_retl_prc_locl          float64
 10  price_group                  object 
 11  pos_disc_amt_locl            float64
 12  ordr_net_sale_retl_amt_locl  float64
 13  ordr_net_sale_cost_amt_locl  float64
 14  trans_dt                     object 
 15  treatment_campaign_id        int64  
 16  name_email_campaign          object 
 17  treatment_id                 int64  
 18  name_email_category          object 
 19  

In [ ]:
df.isnull().sum()

,0
date,0
fullvisitor_id,0
transaction_id,0
product_sku,0
quantity,0
product_revenue,0
country,0
retn_flag,0
disc_offr_id,0
orgnl_retl_prc_locl,3767


## Save notebook to GCS

In [ ]:
BUCKET_NAME = "mma-capstone"
NOTEBOOK_PATH = "/content/drive/MyDrive/Colab Notebooks/UC2_Merge_Email_Campaigns.ipynb"
DESTINATION_BLOB_NAME = "notebooks/UC2_Merge_Email_Campaigns.ipynb"


client = storage.Client()
bucket = client.bucket(BUCKET_NAME)
blob = bucket.blob(DESTINATION_BLOB_NAME)
blob.upload_from_filename(NOTEBOOK_PATH)

print(f"Notebook saved to GCS: gs://{BUCKET_NAME}/{DESTINATION_BLOB_NAME}")


Notebook saved to GCS: gs://mma-capstone/notebooks/UC2_Merge_Email_Campaigns.ipynb
